In [1]:
from OddoDownload import OddoDownload
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
conn_params = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}
odoo = OddoDownload(conn_params)

In [3]:
# =========================
# DESCARGAR TABLA DE VENTAS
# =========================


modelo = 'x_ventas'
filtros = ['&',('x_studio_unidades_de_negocio','=','SMK'),('x_studio_periodo.x_name','=',2023)]
campos = ['x_studio_producto','x_studio_descripcin_producto','x_studio_elementos_del_producto','x_studio_total_conveniencia',
          'x_studio_total_jumbo','x_studio_total_sisa','x_studio_total_easy','x_studio_total_paris']

header = ['Producto','Producto/Descripción','lista_elementos',
          'TOTAL CONVENIENCIA','TOTAL JUMBO','TOTAL SISA','TOTAL EASY','TOTAL PARIS']

campos_fk = ['x_studio_producto']


partes = odoo.getDataFromModel(modelo,filtros,campos,ret_=True,campos_fk=campos_fk, header=header)
partes.head()

Descargando data desde x_ventas
.
done


,Producto,Producto/Descripción,lista_elementos,TOTAL CONVENIENCIA,TOTAL JUMBO,TOTAL SISA,TOTAL EASY,TOTAL PARIS
0,41842SMK,SELLO SEGURIDAD COLORES PLASTICO JUMBO,[1083569],0.0,37000.0,0.0,0.0,0.0
1,42502SMK,ENVASE POLLO CON TAPA.,[1042963],183400.0,17000.0,0.0,0.0,0.0
2,42503SMK,ESTUCHE 143 PP PASTA MICROONDA,[1042960],58200.0,382400.0,0.0,0.0,0.0
3,43595SMK,SELLOS SEG CELESTE CONTROL INTERNO SISA,[1083570],39000.0,0.0,0.0,0.0,0.0
4,43596SMK,SELLO SES/20 AZUL NUM.CORREL LOG/JUMBO,[1083571],1000.0,64000.0,0.0,0.0,0.0


In [4]:
# =========================
# CONTAR PARTE A DESCARGAR
# =========================

total_elementos = []
for i in range(len(partes)):
    elementos = eval(partes['lista_elementos'].iloc[i])
    total_elementos += elementos
len(set(total_elementos))

135788

In [5]:
# ===========================
# DESCARGAR DETALLE DE PARTES
# ===========================


modelo = 'x_materialidad'
filtros = [("id","in",total_elementos)]
campos = ['x_name','x_studio_productos_por_envase','x_studio_peso','x_studio_peso_informado','x_studio_mat',
          'x_studio_caractertica_del_material_solo_para_plsticos','x_studio_definir_otro_material','x_studio_caracterstica_reciclable',
          'x_studio_caracteristica_retornable','x_studio_peligrosidad','x_studio_categora','x_studio_cat_material']

header = ['Elementos del producto','Productos por envase','Peso','Peso informado','Material','caracterítica del material (solo para plásticos, cartón y vidrio)',
          'Definir otro material','Característica reciclable','Característica retornable','Peligrosidad','Categoría','Categoría material']

campos_fk = ['x_studio_mat']

materialidad = odoo.getDataFromModel(modelo,filtros,campos,ret_=True,drop_id=False,campos_fk=campos_fk,header=header)
print('registros descargados:',len(materialidad))
materialidad['id'] = pd.to_numeric(materialidad['id'], errors='coerce')
materialidad.head()


Descargando data desde x_materialidad
.
.
.
done
registros descargados: 135788


,id,Elementos del producto,Productos por envase,Peso,Peso informado,Material,"caracterítica del material (solo para plásticos, cartón y vidrio)",Definir otro material,Característica reciclable,Característica retornable,Peligrosidad,Categoría,Categoría material
0,64074,TAPE,36,0.5555555555555556,20.0,Envases de PP que NO contienen sustancias con ...,Flexible,False,Sin material reciclado,No Retornable,Residuo NO Peligroso,EYE No domiciliario,PLÁSTICOS
1,64204,LABEL,1,1.0,1.0,Papel,False,False,Sin material reciclado,No Retornable,Residuo NO Peligroso,EYE Domiciliario,PAPEL Y CARTÓN
2,64205,BOX,24,41.666666666666664,1000.0,CARTÓN,Corrugado,False,Sin material reciclado,No Retornable,Residuo NO Peligroso,EYE No domiciliario,PAPEL Y CARTÓN
3,64206,CINTA ADHESIVA,24,0.25,6.0,Envases de PP que NO contienen sustancias con ...,Flexible,False,Sin material reciclado,No Retornable,Residuo NO Peligroso,EYE No domiciliario,PLÁSTICOS
4,64211,LABEL,1,0.3,0.3,Papel,False,False,Sin material reciclado,No Retornable,Residuo NO Peligroso,EYE Domiciliario,PAPEL Y CARTÓN


In [6]:
# ===========================
# CREAR TABLA FINAL
# ===========================

header_1 = ['Producto','Producto/Descripción']
header_2 = ['Elementos del producto','Productos por envase','Peso','Peso informado','Material','caracterítica del material (solo para plásticos, cartón y vidrio)',
          'Definir otro material','Característica reciclable','Característica retornable','Peligrosidad','Categoría','Categoría material']
header_3 = ['TOTAL CONVENIENCIA','TOTAL JUMBO','TOTAL SISA','TOTAL EASY','TOTAL PARIS']
final_header = header_1+header_2+header_3

n_campos = len(final_header)
declaracion_eye = np.zeros( (len(total_elementos),n_campos),dtype='object' )

index_declaracion = 0
for i in tqdm(range(len(partes))):
    lista_elementos = eval(partes.iloc[i].lista_elementos)
    parte1 = partes[header_1].iloc[i].to_numpy().reshape(1,-1)  
    parte3 = partes[header_3].iloc[i].to_numpy().reshape(1,-1)

    for elemento in lista_elementos:
        detalle_elemento = materialidad[ materialidad['id']==elemento ]
        detalle_elemento = detalle_elemento[header_2].to_numpy().reshape(1,-1)
        row_declaracion = np.concatenate([parte1,detalle_elemento,parte3],axis=1)

        declaracion_eye[index_declaracion] = row_declaracion
        index_declaracion += 1

declaracion_eye = pd.DataFrame(data=declaracion_eye,columns=final_header)
declaracion_eye = declaracion_eye[(declaracion_eye['Categoría']=='EYE Domiciliario') | (declaracion_eye['Categoría']=='EYE No domiciliario')]

    

100%|██████████| 22218/22218 [01:21<00:00, 272.31it/s]


In [10]:
declaracion_eye.to_csv('Declaracion eye SMK.csv')